In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from IPython.display import Audio
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

## Dataset

In [ ]:
path = "../speech_emotion_dataset"
# audio_dir = "./Crema"

In [ ]:
def load_and_listen(audio_path, sample_rate=22050):
    # Load the audio
    y, sr = librosa.load(audio_path, sr=sample_rate)

    # Plot the waveform
    plt.figure(figsize=(14, 4))
    librosa.display.waveshow(y, sr=sr)
    plt.title(f"Waveform of {audio_path}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    plt.show()

    # Play the audio
    return Audio(y, rate=sr)


In [ ]:

# Locate the folder with .wav files
audio_dir = os.path.join(path, "Crema")  # Make sure this folder name matches what you see

# List all .wav files
audio_files = [f for f in os.listdir(audio_dir) if f.endswith(".wav")]
print(f"Total audio files: {len(audio_files)}")

# Mapping of emotion codes to labels
emotion_map = {
    "ANG": "Angry",
    "DIS": "Disgust",
    "FEA": "Fear",
    "HAP": "Happy",
    "NEU": "Neutral",
    "SAD": "Sad"
}

# Extract a sample of one file per emotion
samples = {}
for file in audio_files:
    parts = file.split('_')
    emotion_code = parts[2]
    if emotion_code in emotion_map and emotion_code not in samples:
        samples[emotion_code] = file

# Plot and listen to one file per emotion
for code, filename in samples.items():
    filepath = os.path.join(audio_dir, filename)
    signal, sr = librosa.load(filepath, sr=None)
    
    # Plot waveform
    plt.figure(figsize=(10, 3))
    librosa.display.waveshow(signal, sr=sr)
    plt.title(f"{emotion_map[code]} - {filename}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.tight_layout()
    plt.show()
    
    # Play audio (if running in Jupyter Notebook)
    try:
        from IPython.display import Audio, display
        display(Audio(filepath))
    except:
        pass

In [ ]:
def extract_features(filepath, sr=16000, frame_length=2048, hop_length=512, n_mels=128):
    # Load audio
    signal, sr = librosa.load(filepath, sr=sr)  # Use desired sampling rate
    
    # Trim silent edges
    signal, _ = librosa.effects.trim(signal)
    
    # --- Zero Crossing Rate ---
    zcr = librosa.feature.zero_crossing_rate(
        y=signal, frame_length=frame_length, hop_length=hop_length
    )[0]  # shape: (frames,)

    # --- Energy (normalized) ---
    energy = np.array([
        np.sum(signal[i:i+frame_length]**2) / frame_length
        for i in range(0, len(signal) - frame_length + 1, hop_length)
    ])
    
    # --- Mel Spectrogram ---
    mel_spec = librosa.feature.melspectrogram(
        y=signal, sr=sr, n_mels=n_mels,
        n_fft=frame_length, hop_length=hop_length
    )

    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    return zcr, energy, mel_spec_db

In [ ]:
# Test on a sample
test_file = os.path.join(audio_dir, random.choice(audio_files))
zcr, energy, mel_spec = extract_features(test_file )
fs1 = zcr , energy
fs2 = mel_spec
# Plot features
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(zcr)
plt.title("Zero Crossing Rate")

plt.subplot(1, 3, 2)
plt.plot(energy)
plt.title("Energy")

plt.subplot(1, 3, 3)
librosa.display.specshow(mel_spec, sr=16000, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title("Mel Spectrogram")

plt.tight_layout()
plt.show()

In [ ]:
# Prepare containers
zcr_list = []
energy_list = []
mel_spec_list = []
labels = []

valid_emotions = {"ANG", "DIS", "FEA", "HAP", "NEU", "SAD"}

# Loop through files
for file in tqdm(audio_files):
    parts = file.split('_')
    emotion_code = parts[2]

    if emotion_code not in valid_emotions:
        continue  # skip unknown emotions

    filepath = os.path.join(audio_dir, file)
    
    try:
        zcr, energy, mel_spec = extract_features(filepath)
        
        zcr_list.append(zcr)
        energy_list.append(energy)
        mel_spec_list.append(mel_spec)
        labels.append(emotion_code)
        
    except Exception as e:
        print(f"Failed for {file}: {e}")

In [ ]:
X =zcr_list, energy_list, mel_spec_list
print(len(zcr_list), len(energy_list), len(mel_spec_list))

le = LabelEncoder()
label_array = le.fit_transform(labels)

# Get mapping
label_map = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label mapping:", label_map)
label_array.shape
print(label_array.shape)

# Convert to DataFrame
df = pd.DataFrame({
    'zcr': zcr_list,
    'energy': energy_list,
    'mel_spec': mel_spec_list,
    'label': label_array
})

X = df.drop(columns=['label'])
y = df['label']

print("Shape of features:", X.shape)
print("Shape of labels:", y.shape)


### Splitting the data

In [ ]:
# 1. Train+Val (70%) and Test (30%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

# 2. From Train+Val, get 5% as validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.05,  # 5% of 70% ≈ 3.5% of total data
    stratify=y_temp,
    random_state=42
)

# Show shapes
print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)

### Create the feature space

In [ ]:
"""First feature Space""" 
X_train1 = X_train[X_train.columns[0:2]]
X_val1   = X_val[X_val.columns[0:2]]
X_test1  = X_test[X_test.columns[0:2]]

"""Second feature space"""
X_train2 = X_train[X_train.columns[2:]]
X_val2   = X_val[X_val.columns[2:]]
X_test2  = X_test[X_test.columns[2:]]

print("Train shape1:", X_train1.shape)
print("Validation shape1:", X_val1.shape)
print("Test shape1:", X_test1.shape)

print("Train shape2:", X_train2.shape)
print("Validation shape2:", X_val2.shape)
print("Test shape2:", X_test2.shape)

### Prepare Data for Model Input

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad 1D features
def stack_and_pad_1d(features1, features2, max_len=400):
    x1 = pad_sequences(features1, maxlen=max_len, padding='post', dtype='float32')
    x2 = pad_sequences(features2, maxlen=max_len, padding='post', dtype='float32')
    return np.stack([x1, x2], axis=-1)

X_train1_padded = stack_and_pad_1d(X_train1['zcr'], X_train1['energy'])
X_val1_padded = stack_and_pad_1d(X_val1['zcr'], X_val1['energy'])
X_test1_padded = stack_and_pad_1d(X_test1['zcr'], X_test1['energy'])

# Pad mel spectrograms
def pad_melspec(mels, max_time=400):
    padded = []
    for m in mels:
        if m.shape[1] < max_time:
            pad_width = max_time - m.shape[1]
            m_padded = np.pad(m, ((0, 0), (0, pad_width)), mode='constant')
        else:
            m_padded = m[:, :max_time]
        padded.append(m_padded)
    return np.array(padded)[..., np.newaxis]

X_train2_padded = pad_melspec(X_train2['mel_spec'])
X_val2_padded = pad_melspec(X_val2['mel_spec'])
X_test2_padded = pad_melspec(X_test2['mel_spec'])

# Convert labels
y_train_arr = tf.keras.utils.to_categorical(y_train)
y_val_arr = tf.keras.utils.to_categorical(y_val)
y_test_arr = tf.keras.utils.to_categorical(y_test)


## Define CNN Models

### 1D CNN (ZCR + Energy)

In [ ]:
def build_1d_cnn(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv1D(512, kernel_size=5, strides=1, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same'),
        tf.keras.layers.Conv1D(512, kernel_size=5, strides=1, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same'),
        tf.keras.layers.Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same'),
        tf.keras.layers.Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same'),
        tf.keras.layers.Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=5,strides=2,padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model


### 2D CNN (Mel Spectrogram)

In [ ]:
def build_2d_cnn(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Conv2D(512, (5, 5), strides=1, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(5, 5),strides=2,padding='same'),
        tf.keras.layers.Conv2D(512, (5, 5), strides=1, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(5, 5),strides=2,padding='same'),
        tf.keras.layers.Conv2D(256, (5, 5), strides=1, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(5, 5),strides=2,padding='same'),
        tf.keras.layers.Conv2D(256, (5, 5), strides=1, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(5, 5),strides=2,padding='same'),
        tf.keras.layers.Conv2D(128, (5, 5), strides=1, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(5, 5),strides=2,padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model


### Train the 1D CNN Model

In [ ]:
callbacks = [
    EarlyStopping(monitor= 'val_accuracy', patience=5, restore_best_weights=True, mode ='max'),
    ReduceLROnPlateau(monitor='val_accuracy',patience=3,verbose=1,factor=0.5,min_lr=0.00001)
]

In [ ]:
# Train 1D CNN
model_1d = build_1d_cnn(X_train1_padded.shape[1:], 6)
model_1d.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_1d.summary()
history_1D = model_1d.fit(X_train1_padded, y_train_arr, epochs=100, batch_size=32, 
             validation_data=(X_val1_padded, y_val_arr), callbacks=callbacks)

### Train the 2D CNN Model

In [ ]:
# Train 2D CNN
model_2d = build_2d_cnn(X_train2_padded.shape[1:], 6)
model_2d.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_2d.summary()
history_2D = model_2d.fit(X_train2_padded, y_train_arr, epochs=30, batch_size=32, 
             validation_data=(X_val2_padded, y_val_arr), callbacks=callbacks)


## Models Evaluation

In [ ]:
# train accuracy
train_acc_1d = history_1D.history['accuracy']
train_acc_2d = history_2D.history['accuracy']
# validation accuracy
val_acc_1d = history_1D.history['val_accuracy']
val_acc_2d = history_2D.history['val_accuracy']
# train loss
train_loss_1d = history_1D.history['loss']
train_loss_2d = history_2D.history['loss']
# validation loss
val_loss_1d = history_1D.history['val_loss']
val_loss_2d = history_2D.history['val_loss']

print("Train accuracy 1D:", train_acc_1d)
print("Validation accuracy 1D:", val_acc_1d)

print("Train accuracy 2D:", train_acc_2d)
print("Validation accuracy 2D:", val_acc_2d)

# Plotting
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_acc_1d, label='Train 1D')
plt.plot(val_acc_1d, label='Validation 1D')
plt.plot(train_acc_2d, label='Train 2D')
plt.plot(val_acc_2d, label='Validation 2D')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.legend()
plt.subplot(1, 2, 2)
plt.plot(train_loss_1d, label='Train 1D')
plt.plot(val_loss_1d, label='Validation 1D')
plt.plot(train_loss_2d, label='Train 2D')
plt.plot(val_loss_2d, label='Validation 2D')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()
